In [1]:
import ipywidgets as widgets
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint
import gmaps.geojson_geometries
from IPython.display import display
import ipywidgets as widgets
import datetime as dt
from sodapy import Socrata

from config import api_key, key_dict

gkey = key_dict.get('maps', api_key)

Code to create map with LA neighborhood coordinates. Covid data is per neighborhood.
Build dataframe with neighborhoods and covid rates

In [2]:
cov_path = os.path.join("Resources", "Covid-19.csv")
pop_path = os.path.join("Resources", "LApop.csv")
neighgeo = os.path.join("Resources", "la-county-municipal-regions-current.geojson")

In [3]:
covid = pd.read_csv(cov_path, encoding="utf-16")
census = pd.read_csv(pop_path)

In [4]:
census2 = census.rename(columns={'Neighborhood':'Region'})
census2

,Policy Area,Dataset,Variable,Year,Count,Tract,Tract Number,Region,GEOID,Location,Row ID,Date
0,Demography,Total Population,Total Population,2019,732.0,Census Tract 8003.31,800331,Westlake Village,1400000US06037800331,"(34.1572764, -118.79983879)",Total_Population_2019_1400000US06037800331,01/01/2019
1,Demography,Total Population,Total Population,2019,6219.0,Census Tract 2171,217100,Mid-Wilshire,1400000US06037217100,"(34.0489706, -118.35454176)",Total_Population_2019_1400000US06037217100,01/01/2019
2,Demography,Total Population,Total Population,2019,4811.0,Census Tract 2716,271600,Mar Vista,1400000US06037271600,"(34.02121853, -118.43107578)",Total_Population_2019_1400000US06037271600,01/01/2019
3,Demography,Total Population,Total Population,2019,3903.0,Census Tract 1094,109400,Mission Hills,1400000US06037109400,"(34.26872263, -118.45570915)",Total_Population_2019_1400000US06037109400,01/01/2019
4,Demography,Total Population,Total Population,2019,2744.0,Census Tract 9010.08,901008,Lancaster,1400000US06037901008,"(34.69664725, -118.17482302)",Total_Population_2019_1400000US06037901008,01/01/2019
...,...,...,...,...,...,...,...,...,...,...,...,...
2339,Demography,Total Population,Total Population,2019,6550.0,Census Tract 1134.01,113401,Winnetka,1400000US06037113401,"(34.22723213, -118.56270602)",Total_Population_2019_1400000US06037113401,01/01/2019
2340,Demography,Total Population,Total Population,2019,4516.0,Census Tract 3018.02,301802,Glendale,1400000US06037301802,"(34.14921893, -118.25971473)",Total_Population_2019_1400000US06037301802,01/01/2019
2341,Demography,Total Population,Total Population,2019,3910.0,Census Tract 4812.01,481201,Temple City,1400000US06037481201,"(34.09762259, -118.07110837)",Total_Population_2019_1400000US06037481201,01/01/2019
2342,Demography,Total Population,Total Population,2019,4577.0,Census Tract 5730.04,573004,Long Beach,1400000US06037573004,"(33.79349546, -118.19648557)",Total_Population_2019_1400000US06037573004,01/01/2019


In [5]:
census3 = census2.drop(['Policy Area', 'Dataset','Variable','Year','Tract','Tract Number','GEOID','Location','Row ID','Date'], axis=1)

In [6]:
census4 = census3.drop(census3[census3.Region.isin(['Elysian Park', 'Vernon','Harvard Park'])].index)

In [7]:
census4.head()

,Count,Region
0,732.0,Westlake Village
1,6219.0,Mid-Wilshire
2,4811.0,Mar Vista
3,3903.0,Mission Hills
4,2744.0,Lancaster


In [13]:
popneigh = census4.groupby(['Region'])

In [15]:
neigh = popneigh['Region'].first()
neighsum = popneigh['Count'].sum()

In [24]:
popcount = pd.DataFrame({'Population':neighsum})

In [25]:
popcount.head(20)

,Population
Region,
Acton,7686.0
Adams-Normandie,18255.0
Agoura Hills,19336.0
Agua Dulce,4080.0
Alhambra,84647.0
Alondra Park,4777.0
Altadena,44166.0
Angeles Crest,886.0
Arcadia,55243.0


In [26]:
popcount.reset_index(level=0, inplace=True)

In [27]:
popcount.head()

,Region,Population
0,Acton,7686.0
1,Adams-Normandie,18255.0
2,Agoura Hills,19336.0
3,Agua Dulce,4080.0
4,Alhambra,84647.0


In [11]:
covid.head()

,Time Stamp,Region,Latitude,Longitude,Number of cases
0,01-1-2021,Acton,34.480742,-118.186838,271
1,01-1-2021,Adams-Normandie,34.031788,-118.300247,766
2,01-1-2021,Agoura Hills,34.147910,-118.765704,593
3,01-1-2021,Alhambra,34.093042,-118.127060,4241
4,01-1-2021,Alsace,33.988000,-118.347620,1016


In [12]:
covid['Time Stamp'].nunique()

368

In [13]:
covid['Region'].nunique()



246

In [32]:
covid['Time Stamp'] = pd.to_datetime(covid['Time Stamp'])
covid.head()


,Time Stamp,Region,Latitude,Longitude,Number of cases
0,2021-01-01,Acton,34.480742,-118.186838,271
1,2021-01-01,Adams-Normandie,34.031788,-118.300247,766
2,2021-01-01,Agoura Hills,34.147910,-118.765704,593
3,2021-01-01,Alhambra,34.093042,-118.127060,4241
4,2021-01-01,Alsace,33.988000,-118.347620,1016


In [33]:
covid['Time'] = covid['Time Stamp'].astype(int)
covid.head()

,Time Stamp,Region,Latitude,Longitude,Number of cases,Time
0,2021-01-01,Acton,34.480742,-118.186838,271,1609459200000000000
1,2021-01-01,Adams-Normandie,34.031788,-118.300247,766,1609459200000000000
2,2021-01-01,Agoura Hills,34.147910,-118.765704,593,1609459200000000000
3,2021-01-01,Alhambra,34.093042,-118.127060,4241,1609459200000000000
4,2021-01-01,Alsace,33.988000,-118.347620,1016,1609459200000000000


In [34]:
covid = covid.sort_values(by='Time Stamp')
covid[covid['Time Stamp'] == "2020-3-16"]
day1 = 1584316800000000000

In [35]:
covid = covid.sort_values(by='Time Stamp')
covid[covid['Time Stamp'] == "2020-3-17"]
day2 = 1584403200000000000

In [36]:
day = day2 - day1
day

86400000000000

In [37]:
covid['Day'] = (covid['Time'] - day1)/day
covid

,Time Stamp,Region,Latitude,Longitude,Number of cases,Time,Day
15729,2020-03-16,Torrance,33.835849,-118.340629,2,1584316800000000000,0.0
15711,2020-03-16,Diamond Bar,34.028623,-117.810337,2,1584316800000000000,0.0
15716,2020-03-16,Granada Hills,34.266156,-118.517434,3,1584316800000000000,0.0
15715,2020-03-16,Glendora,34.136119,-117.865339,1,1584316800000000000,0.0
15714,2020-03-16,Glendale,34.146942,-118.247847,2,1584316800000000000,0.0
...,...,...,...,...,...,...,...
16535,2021-03-18,Winnetka,34.205883,-118.570934,6629,1616025600000000000,367.0
16420,2021-03-18,Lynwood,33.924831,-118.202415,13106,1616025600000000000,367.0
16524,2021-03-18,West Vernon,34.007135,-118.225250,9964,1616025600000000000,367.0
16525,2021-03-18,West Whittier/Los Nietos,34.036211,-118.219955,4151,1616025600000000000,367.0


In [38]:
covidcount = pd.merge(covid, popcount, on='Region')

In [39]:
covidcount.head()

,Time Stamp,Region,Latitude,Longitude,Number of cases,Time,Day,Population
0,2020-03-16,Torrance,33.835849,-118.340629,2,1584316800000000000,0.0,145492.0
1,2020-03-17,Torrance,33.835849,-118.340629,1,1584403200000000000,1.0,145492.0
2,2020-03-18,Torrance,33.835849,-118.340629,2,1584489600000000000,2.0,145492.0
3,2020-03-19,Torrance,33.835849,-118.340629,2,1584576000000000000,3.0,145492.0
4,2020-03-20,Torrance,33.835849,-118.340629,2,1584662400000000000,4.0,145492.0


In [40]:
covidcount.sort_values(by=['Day'])

,Time Stamp,Region,Latitude,Longitude,Number of cases,Time,Day,Population
0,2020-03-16,Torrance,33.835849,-118.340629,2,1584316800000000000,0.0,145492.0
1104,2020-03-16,Glendora,34.136119,-117.865339,1,1584316800000000000,0.0,55146.0
1467,2020-03-16,Glendale,34.146942,-118.247847,2,1584316800000000000,0.0,200232.0
1835,2020-03-16,Gardena,33.896359,-118.305304,1,1584316800000000000,0.0,59709.0
2203,2020-03-16,Boyle Heights,34.043689,-118.209768,5,1584316800000000000,0.0,95358.0
...,...,...,...,...,...,...,...,...
58934,2021-03-18,Rosemead,34.067617,-118.087976,4856,1616025600000000000,367.0,47907.0
25964,2021-03-18,Lakewood,33.850346,-118.117191,7040,1616025600000000000,367.0,83312.0
26328,2021-03-18,Redondo Beach,33.845591,-118.388677,2616,1616025600000000000,367.0,67423.0
23048,2021-03-18,Tujunga,34.252225,-118.288410,2844,1616025600000000000,367.0,25105.0


In [42]:
covidcount['CovPct'] = (covidcount['Number of cases']/covidcount['Population'])*100

In [24]:
covidcount

,Time Stamp,Region,Latitude,Longitude,Number of cases,Time,Day,Count,CovPct
0,2020-03-16,Torrance,33.835849,-118.340629,2,1584316800000000000,0.0,145492.0,0.001375
1,2020-03-17,Torrance,33.835849,-118.340629,1,1584403200000000000,1.0,145492.0,0.000687
2,2020-03-18,Torrance,33.835849,-118.340629,2,1584489600000000000,2.0,145492.0,0.001375
3,2020-03-19,Torrance,33.835849,-118.340629,2,1584576000000000000,3.0,145492.0,0.001375
4,2020-03-20,Torrance,33.835849,-118.340629,2,1584662400000000000,4.0,145492.0,0.001375
...,...,...,...,...,...,...,...,...,...
64857,2021-03-14,Downtown,34.042849,-118.247673,3842,1615680000000000000,363.0,53398.0,7.195026
64858,2021-03-15,Downtown,34.042849,-118.247673,3844,1615766400000000000,364.0,53398.0,7.198771
64859,2021-03-16,Downtown,34.042849,-118.247673,3844,1615852800000000000,365.0,53398.0,7.198771
64860,2021-03-17,Downtown,34.042849,-118.247673,3842,1615939200000000000,366.0,53398.0,7.195026


In [43]:
covidcount['Number of cases'].max()

24637

In [44]:
covidcount.to_csv('covidcountdataframe.csv')

In [27]:
coordinates = covid.loc[covid.Time == 1584316800000000000,:][['Latitude', 'Longitude']]

In [45]:
covidcount.count()

Time Stamp         64862
Region             64862
Latitude           64862
Longitude          64862
Number of cases    64862
Time               64862
Day                64862
Population         64862
CovPct             64862
dtype: int64

In [46]:
covidcount['CovPct'].max()

34.31432223507213

In [47]:
covidcount.sort_values(by=['CovPct'], ascending=False)


,Time Stamp,Region,Latitude,Longitude,Number of cases,Time,Day,Population,CovPct
51080,2021-03-18,Florence-Firestone,33.967426,-118.243307,22550,1616025600000000000,367.0,65716.0,34.314322
51079,2021-03-17,Florence-Firestone,33.967426,-118.243307,22540,1615939200000000000,366.0,65716.0,34.299105
51078,2021-03-16,Florence-Firestone,33.967426,-118.243307,22532,1615852800000000000,365.0,65716.0,34.286932
51077,2021-03-15,Florence-Firestone,33.967426,-118.243307,22526,1615766400000000000,364.0,65716.0,34.277801
51076,2021-03-14,Florence-Firestone,33.967426,-118.243307,22523,1615680000000000000,363.0,65716.0,34.273236
...,...,...,...,...,...,...,...,...,...
30720,2020-04-27,Bellflower,33.882570,-118.116768,0,1587945600000000000,42.0,77195.0,0.000000
23787,2020-03-27,Vermont Knolls,33.966819,-118.291670,0,1585267200000000000,11.0,26284.0,0.000000
49300,2020-03-31,Cudahy,33.962058,-118.183539,0,1585612800000000000,15.0,23890.0,0.000000
49299,2020-03-30,Cudahy,33.962058,-118.183539,0,1585526400000000000,14.0,23890.0,0.000000


In [32]:
#url = 'https://usc.data.socrata.com/resource/y65w-5vdw.json?$$app_token=fJezqntoLQgU80nYtTPxcQcSK'
#censusresults = requests.get(url)
#census_json = censusresults.json()
#pprint(census_json)

In [33]:
#census_df = pd.DataFrame()
#census_df['Neighborhood'] = ''
#census_df['Population'] = ''

In [34]:
#neighborhood = []
#population = []
#for i in census_json:
    #neighborhood.append(i['neighborhood'])
    #population.append(i['count'])
#print(len(neighborhood))

In [48]:
gmaps.configure(api_key=gkey)

In [67]:
class LAneighExplorer(object):
    """
    Jupyter widget for exploring the ACLED dataset.

    The user uses the slider to choose a year. This renders
    a heatmap of civilian victims in that year.
    """

    def __init__(self, covidcount):
        self._covidcount = covidcount
        self._heatmap = None
        self._slider = None
        initial_day = min(self._covidcount['Day'])

        title_widget = widgets.HTML(
            '<h3>Covid % of population in LA per neighborhood per day</h3>'
        )

        map_figure = self._render_map(initial_day)
        controls = self._render_controls(initial_day)
        self._container = widgets.VBox([title_widget, controls, map_figure])

    def render(self):
        display(self._container)

    def _on_day_change(self, change):
       
        day = self._slider.value
        self._heatmap.locations = self._coords_for_day(day)
        self._heatmap.weights = self._weights_for_day(day)
        self._total_box.value = self._average_pospct_text_for_day(day)
        self._fig.add_layer(self._heatmap)
        return self._container

    
    
    def _render_map(self, initial_day):
        self._fig = gmaps.figure(map_type='ROADMAP',layout={'width': '800px', 'height':'500px'}, center=(33.9,-118.2437), zoom_level=10)
        self._heatmap = gmaps.heatmap_layer(self._coords_for_day(initial_day), self._weights_for_day(initial_day), 
                                            max_intensity=5, point_radius=20)
                                            #float(self._weights_for_day(initial_day).max())
        self._fig.add_layer(self._heatmap)
        return self._fig

    def _render_controls(self, initial_day):
        self._slider = widgets.IntSlider(
            value=initial_day,
            min=min(self._covidcount['Day']),
            max=max(self._covidcount['Day']),
            description='Day',
            continuous_update=False
        )
        self._total_box = widgets.Label(
            value=self._average_pospct_text_for_day(initial_day)
        )
        self._slider.observe(self._on_day_change, names='value')
        controls = widgets.HBox(
            [self._slider, self._total_box],
            layout={'justify_content': 'space-between'}
        )
        return controls

    def _weights_for_day(self, day):
        return self._covidcount[self._covidcount['Day'] == day]['CovPct']
        
        

    def _average_pospct_for_day(self, day):
        return round(float(self._covidcount[self._covidcount['Day'] == day]['CovPct'].mean()),2)
    
    def _coords_for_day(self, day):
        return (self._covidcount[self._covidcount['Day'] == day][['Latitude','Longitude']])
        
        
    def _average_pospct_text_for_day(self, day):
        return '{} average % of population with covid'.format(self._average_pospct_for_day(day))

  

LAneighExplorer(covidcount).render()



In [37]:
map_layout = {'width': '500px', 'height':'500px'}
fig = gmaps.figure(layout=map_layout, center=(33.9,-118.2437), zoom_level=10)
fig

Figure(layout=FigureLayout(height='500px', width='500px'))